In [8]:
from langchain_community.tools import  WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [16]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)


In [13]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_google_genai import  GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
from dotenv import load_dotenv
load_dotenv()
embedding_function = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",  # Correct embedding model
    google_api_key=os.getenv('GEMINI_KEY')
)

loader = WebBaseLoader('https://docs.smith.langchain.com/')
docs =loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectordb = FAISS.from_documents(documents, embedding_function)
retriver = vectordb.as_retriever()
retriver

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f2d81fd7e80>, search_kwargs={})

In [14]:
from langchain.tools.retriever import create_retriever_tool
retriver_tool = create_retriever_tool(retriver, 'langsmith_search',
                      "Search for information about Langsmith. For any question search in page")
retriver_tool.name

'langsmith_search'

In [15]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiwraps = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=100)
arxiv = ArxivQueryRun(api_wrapper=arxiwraps)
arxiv.name

'arxiv'

In [21]:
tools=[wiki, arxiv, retriver_tool]

In [17]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os 
load_dotenv()
GOOGLE_API_KEY= os.getenv('GEMINI_KEY')
llm = ChatGoogleGenerativeAI(model='gemini-2.0-flash', google_api_key=GOOGLE_API_KEY)


In [19]:
from langchain import hub
prompt=hub.pull('hwchase17/openai-functions-agent')
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [23]:
##create an agent 
from langchain.agents import create_openai_tools_agent
agent =create_openai_tools_agent(llm, tools, prompt)


In [24]:
##agent executor
from langchain.agents import AgentExecutor
agent_exxecutor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_exxecutor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [25]:
agent_exxecutor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'Tell me about Langsmith'}`


LangSmith FAQsCan I use LangSmith if I don’t use LangChain or LangGraph?

Yes! Many companies who don’t build with LangChain/LangGraph use LangSmith. You can log traces to LangSmith via the Python SDK, the TypeScript SDK, or the API. See here for more information.How easy is it to start using LangSmith if I use LangChain or LangGraph?

Getting started on LangSmith requires just two environment variables in your LangChain or LangGraph code. See how to send traces from your LangGraph agent or your LangChain app.My application isn’t written in Python or TypeScript. Will LangSmith be helpful?

Yes, you can log traces to LangSmith using a standard OpenTelemetry client to access all LangSmith features, including tracing, running evals, and prompt engineering. See the docs.How can LangSmith help with observability and evaluation?

LangSmith






























Products

{'input': 'Tell me about Langsmith',
 'output': "LangSmith is a developer platform designed for every stage of the LLM-powered application lifecycle. It helps you debug, collaborate, test, and monitor your LLM applications, whether you're building with LangChain or not. It allows you to track the inputs and outputs of each step, run evaluations, and monitor cost, latency, and quality. You can also collect user feedback and filter and inspect anomalies. It can be used with Python, Typescript, or via an OpenTelemetry client. If you have an enterprise plan, you can self-host LangSmith."}